<center><img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="30%" /></center>

<hr style="height:2px;border:none"/>
<h1 align='center'> ELI-349 Operación de Sistemas Eléctricos de Potencia </h1>

<h2 align='center'><b>Tarea Final: LOLIN-OPF</b></h2>
<h3 align='center'>Preparado por Miguel Huerta</h3>



<hr style="height:2px;border:none"/>


In [1]:
import os, getpass
from platform import python_version
try: 
  cwd = os.getcwd()
  ls  = os.listdir("C:\\")
  for gu in ls: 
    if 'gurobi' in gu: 
      os.chdir("C:\\"+gu+'\win64\python'+python_version()[:-2].replace('.','')+'\lib\gurobipy')
      import gurobipy as gp
      from gurobipy import GRB
  os.chdir(cwd)
except:
  pass

from time import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from numpy import flatnonzero as find
from scipy.sparse import csr_matrix as sparse
jj=np.sqrt(-1, dtype=complex)

def table(data, nx, ny, unit='(MW)', typeE='P'):
    print(("{:>4s}".format('t(h)')+" {:>13s}"*nx).format(*[typeE+str(k+1)+unit for k in range(nx)]))
    for k in range(ny):
        print(("{:>4d}".format(k)+" {:>13.3f}"*nx).format(*data[:,k]))
    print()

Archivos suminstrados
- case39.pycase39.py
- Data SEP.xlsData SEP.xls
- Tarea 2021.pdfTarea 2021.pdf
- Valle de los vientos.txtValle de los vientos.txt

## Parte 1
Desarrolle la formulación matemática del problema del UC aplicado al siguiente sistema de 
potencia considerando la temática que su grupo eligió. Note que en el SEP se han incluido los 
datos del sistema de transmisión en p.u. y el límite térmico de cada línea (MVA). En la figura 
también se muestra los valores de demanda en MVA de la hora t. Las cargas tienen un factor de 
potencia de 0.9 inductivo. Para la formulación DC la demanda aproxime a MW, es decir factor 
de potencia de 1. Modele un horizonte de 3 horas, donde la demanda para la segunda hora es 
el 80% y para la tercera hora es del 50% de la demanda.

<center><img src="FIgs\1.PNG" title="Title text" width="30%" /></center>

In [2]:
## Loads
kLoad = [1, .8, .5]                    # Escala de potencia en horizonte de simulación
PF    = [0.9, 1.0]                     # Factores de potencia ante las situaciones a simular
Load  = np.array([[3, 100],[4, 200]])  # Magnitud de potencia en barra j
nh    = len(kLoad)

## Lines
Aux = np.eye(2)-np.flipud(np.eye(2))
LineProp = np.array([[1, 2, 0.08, 0.10, 100],
                     [1, 3, 0.05, 0.10,  80],
                     [2, 3, 0.05, 0.30, 100],
                     [2, 4, 0.04, 0.15, 100],
                     [3, 4, 0.07, 0.15, 100],]) # no hay susceptancia
nl = LineProp.shape[0]
zline = LineProp[:, 2]+jj*LineProp[:, 3]
yline = 1/zline
shap = len(np.unique(LineProp[:,:2]))
Ybus = np.zeros([shap,shap], dtype=complex)
for num, k in enumerate(LineProp[:,:2].astype(int)):
    Ybus[np.ix_(k-1 ,k-1 )] += Aux*yline[num]
print(np.around(Ybus,3))

[[ 8.878-14.098j -4.878 +6.098j -4.    +8.j     0.    +0.j   ]
 [-4.878 +6.098j  7.078-15.565j -0.541 +3.243j -1.66  +6.224j]
 [-4.    +8.j    -0.541 +3.243j  7.095-16.718j -2.555 +5.474j]
 [ 0.    +0.j    -1.66  +6.224j -2.555 +5.474j  4.214-11.699j]]


Los datos técnico-económicos del sistema de generación son los siguientes:


$$\begin{array}{|c|cccc|}\hline
 & 1& 2& 3& 4 \\\hline
P_{max}, MW  & 100& 120& 150& 100\\
P_{min}, MW  &  10&  50&  40&  10\\
CVP~\$/MWh      &  60& 100& 190&  90\\
Q_{max}, Mvar&-100&-100&-100&-100\\
Q_{min}, Mvar& 100& 100&  50&  50\\
\hline\end{array}$$

- En este problema inicial no se consideran los costos de encendido y las restricciones de 
acoplamiento temporal de generación (tiempos mínimos y rampas).
- Modele los generadores virtuales (energía no suministrada). Para modelar la pérdida de carga 
utilice la demanda máxima de la barra y un costo de 500 $/MWh.
- Justifique en el caso que requiera algún parámetro adicional

In [52]:
## Generator       bus  Pmax Pmin  Qmax Qmin  Costes Lineales
GenProp = np.array([[1,  100,  10, 100, 100,  60],
                    [2,  120,  50, 100, 100, 100],
                    [2,  150,  40, 100,  50, 190],
                    [4,  100,  10, 100,  50,  90],])

# Equipos Viertuales donde hay carga
GenProp = np.vstack([GenProp,
          np.hstack([Load,np.zeros([Load.shape[1],1]),Load[:,1].reshape(-1,1),np.zeros([Load.shape[1],1]), np.ones([Load.shape[1],1])*500])
])
ng = GenProp.shape[0]

GenProp

array([[  1., 100.,  10., 100., 100.,  60.],
       [  2., 120.,  50., 100., 100., 100.],
       [  2., 150.,  40., 100.,  50., 190.],
       [  4., 100.,  10., 100.,  50.,  90.],
       [  3., 100.,   0., 100.,   0., 500.],
       [  4., 200.,   0., 200.,   0., 500.]])

## Unit Commitment: Predespacho de la generación térmica

In [68]:
try:
  t0 = time()
  m = gp.Model("Unit Commitment")
  m.Params.OutputFlag = 0
  m.Params.Method = -1
  m.Params.MIPGap = 1e-6

  # Variable de decisión
  binary = m.addMVar(shape=(ng,nh),   vtype=GRB.BINARY,     name='B')
  Pgen   = m.addMVar(shape=(ng,nh),   vtype=GRB.CONTINUOUS, name='P')

  m.update()
  
  # Función de coste
  m.setObjective(sum([GenProp[:,5]@Pgen[:,h] for h in range(nh)]), GRB.MINIMIZE)
  m.update()
  
  for h in range(nh): 
    # Restricción de balance
    m.addConstr( Pgen[:,h].sum()==Load[:,-1].sum()*kLoad[h]*PF[0], name="Balace_"+str(h+1))

    # Restricciones de generación 
    m.addConstr( Pgen[:,h]>= np.diag(GenProp[:,2])@binary[:,h], name="P_min"+str(h+1))
    m.addConstr(-Pgen[:,h]>=-np.diag(GenProp[:,1])@binary[:,h], name="P_max"+str(h+1))
    
    # Restricciones de energia no suministrada
    
    
  m.update()
  t1 = time()
  m.optimize()
  t2 = time()
  print('FO: ',np.float32(m.objVal))
  print('Tiempo Formulación: {:10.5f}ms'.format((t1-t0)*1e3))
  print('Tiempo Resolución:  {:10.5f}ms'.format((t2-t1)*1e3))
  
  
except gp.GurobiError as e:
  print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
  print('Encountered an attribute error')

FO:  48090.0
Tiempo Formulación:   18.97454ms
Tiempo Resolución:     6.98471ms


In [70]:
table(Pgen.x, ng, nh) # P5 y P6 son equipos virtuales en las barras 3 y 4 (barras con demanda)

t(h)        P1(MW)        P2(MW)        P3(MW)        P4(MW)        P5(MW)        P6(MW)
   0       100.000        70.000         0.000       100.000         0.000         0.000
   1       100.000        50.000         0.000        66.000         0.000         0.000
   2       100.000         0.000         0.000        35.000         0.000         0.000



## LOLIN
En base a los datos, se redefinen en función a antiguas implementaciones

In [145]:
Sb = 100
a  = np.zeros(GenProp[:,5].shape)
b  = GenProp[:,5]
c  = np.zeros(GenProp[:,5].shape)
idxSL=0 # barra 1

a1  = np.zeros(GenProp[:,5].shape)
b1  = np.hstack([np.zeros(GenProp[:,5].shape[0]-Load.shape[1]),GenProp[-Load.shape[1]:,5]])
c1  = np.zeros(GenProp[:,5].shape)

# Generation
PGmax = GenProp[:,1]/Sb
PGmin = GenProp[:,2]/Sb
QGmax = GenProp[:,3]/Sb
QGmin = GenProp[:,4]/Sb

# Demand 
PD = Load[:,1]*PF[0]/Sb
QD = Load[:,1]*np.arccos(PF[0])/Sb
PD = np.hstack([np.zeros(2),PD])
QD = np.hstack([np.zeros(2),QD])

# Lines
LineProp[:,0]
fbus = LineProp[:,0].astype(int)-1
tbus = LineProp[:,1].astype(int)-1
Rpu  = LineProp[:,2]
Xpu  = LineProp[:,3]
nb   = int(LineProp[:,0:2].astype(int).max())
Gpu  = np.zeros(nb)
Bpu  = np.zeros(nb)
bpu  = LineProp[:,0]*0

# Transformator
tap    = np.ones(nl)
tapAng = np.zeros(nl)

# # Bus
Vmax = np.ones(nb)*1.05
Vmin = np.ones(nb)*0.95
thetamin = np.ones(nb)*(-np.pi)
thetamax = np.ones(nb)*( np.pi)

yprim = np.diag(1/(Rpu+jj*Xpu))
Yshunt = np.diag(Gpu+jj*Bpu)
bl = jj*bpu

## Potencia maxima lineas
s = LineProp[:,4]/Sb
s[np.where(s==0)]=GRB.INFINITY

# Matriz de incidencia
Cg = sparse((np.ones(ng), (GenProp[:,0]-1, range(ng))), (nb, ng)).toarray().astype(int)
I = np.r_[range(nl), range(nl)]
Cbr = sparse((np.r_[np.ones(nl), -np.ones(nl)], (I, np.r_[fbus, tbus])), (nl,nb)).toarray().astype(int)
Cf  = sparse((np.r_[np.ones(nl), np.zeros(nl)], (I, np.r_[fbus, tbus])), (nl,nb)).toarray().astype(int)
Ct  = sparse((np.r_[np.zeros(nl), np.ones(nl)], (I, np.r_[fbus, tbus])), (nl,nb)).toarray().astype(int)

tapMff = np.diag(1/(tap*np.exp(tapAng)))
tapMft = np.diag(1/(tap**2))
tapMtf = np.diag(1/(tap*np.exp(-tapAng)))

yff  = (yprim+np.diag(bl/2))*tapMff
ytt  = yprim+np.diag(bl/2)
yffp = yprim*tapMft
yttp = yprim*tapMtf
yft  = -yprim*tapMft
ytf  = -yprim*tapMtf

Yf  = yff@Cf+yft@Ct
Yt  = ytf@Cf+ytt@Ct
Yfp = yffp@Cf+yft@Ct
Ytp = ytf@Cf+yttp@Ct
Yb  = Cf.T@Yf+Ct.T@Yt + Yshunt
Ybp = Cf.T@Yfp+Ct.T@Ytp

Aq = np.diag([np.sqrt(2)-1]*nl)
gx  = np.real(yprim)
bx  = -np.imag(yprim)

array([  0.,   0.,   0.,   0., 500., 500.])

In [157]:
k1 = 0.025
k2 = 0.01
try:  
  t0 = time()
  m = gp.Model("LOLIN")
  m.Params.OutputFlag = 0
  m.Params.Method = -1
  m.Params.MIPGap = 1e-6

  # VARIABLE DEFINITIONS
  Pgen  = m.addMVar((ng,nh), vtype=GRB.CONTINUOUS, name='Pg')
  Qgen  = m.addMVar((ng,nh), vtype=GRB.CONTINUOUS, name='Qg')
  Pf    = m.addMVar((nl,nh), vtype=GRB.CONTINUOUS, name='Pf')
  Qf    = m.addMVar((nl,nh), vtype=GRB.CONTINUOUS, name='Qf')
  P_lt  = m.addMVar((nl,nh), vtype=GRB.CONTINUOUS, name='P_lt')
  P_lv  = m.addMVar((nl,nh), vtype=GRB.CONTINUOUS, name='P_lv')
  Q_lt  = m.addMVar((nl,nh), vtype=GRB.CONTINUOUS, name='Q_lt')
  Q_lv  = m.addMVar((nl,nh), vtype=GRB.CONTINUOUS, name='Q_lv')
  V     = m.addMVar((nb,nh), vtype=GRB.CONTINUOUS, name='V')
  theta = m.addMVar((nb,nh), vtype=GRB.CONTINUOUS, name='theta')
  m.update()

  # Función de coste
  CF  = sum([Pgen[:,h]@np.diag(a*Sb*Sb)@Pgen[:,h]+(b*Sb)@Pgen[:,h]+sum(c) for h in range(nh)])
  CF2 = sum([Qgen[:,h]@np.diag(a1*Sb*Sb)@Qgen[:,h]+(b1*Sb)@Qgen[:,h]+sum(c1) for h in range(nh)])
  
  m.setObjective(CF+CF2, GRB.MINIMIZE)
  m.update()
  
  for h in range(nh): 
    # Restricción de balance (26-27)
    m.addConstr(-np.imag(Ybp)@theta[:,h]+np.real(Yb)@V[:,h]-Cg@Pgen[:,h]+np.abs(Cbr).T@P_lt[:,h]+np.abs(Cbr).T@P_lv[:,h] == -(PD)*kLoad[h], name='(26)')
    m.addConstr(-np.real(Ybp)@theta[:,h]-np.imag(Yb)@V[:,h]-Cg@Qgen[:,h]+np.abs(Cbr).T@Q_lt[:,h]+np.abs(Cbr).T@Q_lv[:,h] == -(QD)*kLoad[h], name='(27)')

    m.addConstr( k1*gx@Cbr@theta[:,h]-P_lt[:,h]>=0, name='(a)')
    m.addConstr(-k1*gx@Cbr@theta[:,h]-P_lt[:,h]>=0, name='(b)')

    # m.addConstr( k2*gx@Cbr@V[:,h]-P_lv[:,h]>=0, name='(c)')
    # m.addConstr(-k2*gx@Cbr@V[:,h]-P_lv[:,h]>=0, name='(d)')

    # m.addConstr( k1*bx@Cbr@theta[:,h]-Q_lt[:,h]>=0, name='(e)')
    # m.addConstr(-k1*bx@Cbr@theta[:,h]-Q_lt[:,h]>=0, name='(f)')

    # m.addConstr( k2*bx@Cbr@V[:,h]-Q_lv[:,h]>=0, name='(g)')
    # m.addConstr(-k2*bx@Cbr@V[:,h]-Q_lv[:,h]>=0, name='(h)')
    

    m.addConstr(Pf[:,h] == -np.imag(Yfp)@theta[:,h]+np.real(Yf)@V[:,h], name='(Aux1)')
    m.addConstr(Qf[:,h] == -np.real(Yfp)@theta[:,h]-np.imag(Yf)@V[:,h], name='(Aux2)')
  
    # Bounds
    m.addConstr(   Pf[:,h]+Aq@Qf[:,h]  >= -s, name='B1m')
    m.addConstr( -(Pf[:,h]+Aq@Qf[:,h]) >= -s, name='B1M')

    m.addConstr(   Pf[:,h]-Aq@Qf[:,h]  >= -s, name='B2m')
    m.addConstr( -(Pf[:,h]-Aq@Qf[:,h]) >= -s, name='B2M')

    m.addConstr(   Aq@Pf[:,h]+Qf[:,h]  >= -s, name='B3m')
    m.addConstr( -(Aq@Pf[:,h]+Qf[:,h]) >= -s, name='B3M')

    m.addConstr(   Aq@Pf[:,h]-Qf[:,h]  >= -s, name='B4m')
    m.addConstr( -(Aq@Pf[:,h]-Qf[:,h]) >= -s, name='B4M')
    
    m.addConstr(theta[:,h][idxSL] == 0,'SL') 
    m.addConstr(-Pgen[:,h] >= -PGmax, name='Pmax')
    m.addConstr( Pgen[:,h] >=  PGmin, name='Pmin')
    m.addConstr(-Qgen[:,h] >= -QGmax, name='Qmax')
    m.addConstr( Qgen[:,h] >=  QGmin, name='Qmin')
    m.addConstr(-V[:,h] >= -Vmax, name='Vmax')
    m.addConstr( V[:,h] >=  Vmin, name='Vmin')
    m.addConstr(-theta[:,h] >= -thetamax, name='thetamax')
    m.addConstr( theta[:,h] >=  thetamin, name='thetamin')

  m.update()
  m.optimize()
  print("     FO: {:8.2f}\n".format(m.objVal))
  print(np.hstack([Pgen.x.T, Pgen.x.T.sum(axis=1).reshape(-1,1)]).astype(np.float16), '\n')
  print(np.hstack([Qgen.x.T, Qgen.x.T.sum(axis=1).reshape(-1,1)]).astype(np.float16), '\n')


  
except gp.GurobiError as e:
  print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
  print('Encountered an attribute error')

     FO: 175645.83

[[1.      0.5     0.4     1.      0.5884  0.5903  4.08   ]
 [1.      0.5     0.4     1.      0.4082  0.2302  3.54   ]
 [1.      0.5     0.4     0.8115  0.07385 0.      2.785  ]] 

[[1.    1.    0.5   0.5   0.    0.    3.   ]
 [1.    1.    0.5   0.5   0.    0.    3.   ]
 [1.    1.    0.718 0.5   0.    0.    3.217]] 



array([[1.     , 0.5    , 0.4    , 1.     , 0.5884 , 0.5903 , 4.08   ],
       [1.     , 0.5    , 0.4    , 1.     , 0.4082 , 0.2302 , 3.54   ],
       [1.     , 0.5    , 0.4    , 0.8115 , 0.07385, 0.     , 2.785  ]],
      dtype=float16)

In [63]:
#          Pmax  Pmin Qmax Qmin 
# array([[  1.0,  0.1, 1.0, 1.0,  60.],
#        [  1.2,  0.5, 1.0, 1.0, 100.],
#        [  1.5,  0.4, 1.0, 0.5, 190.],
#        [  1.0,  0.1, 1.0, 0.5,  90.],
#        [  1.0,  0.0, 1.0, 0.0, 500.],
#        [  2.0,  0.0, 2.0, 0.0, 500.]])